# Diffusion Image Editor - Backend on Colab

This notebook runs the FastAPI backend for the Diffusion Image Editor project.

**Setup:**
1. Runtime > Change runtime type > Select **T4 GPU** (or better)
2. Run all cells in order
3. Copy the ngrok URL and paste it in your frontend `.env` file

**VRAM Requirements:**
- SD Inpainting: 5-7 GB
- SDXL Inpainting: 10-12 GB
- CodeFormer/GFPGAN: 2-4 GB
- Real-ESRGAN: 2-6 GB

## 1. Check GPU

In [ ]:
# Check if GPU is available
!nvidia-smi

## 2. Clone Repository

In [ ]:
# Clone the repository
!git clone https://github.com/Yuquan1ee/final_year_project.git
%cd final_year_project/backend

## 3. Install Dependencies

In [ ]:
# Install backend dependencies
!pip install -q fastapi uvicorn[standard] python-multipart pydantic python-dotenv
!pip install -q Pillow opencv-python
!pip install -q torch diffusers transformers accelerate safetensors
!pip install -q gfpgan realesrgan basicsr facexlib

# Install ngrok and nest-asyncio for Colab
!pip install -q pyngrok nest-asyncio

## 4. Setup ngrok

You need an ngrok authtoken to expose the server publicly.

1. Create a free account at https://ngrok.com/
2. Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
3. Paste it below

In [ ]:
# Enter your ngrok authtoken here
NGROK_AUTHTOKEN = ""  # <-- Paste your ngrok authtoken here

if not NGROK_AUTHTOKEN:
    print("WARNING: No ngrok authtoken provided!")
    print("Get your free authtoken at: https://dashboard.ngrok.com/get-started/your-authtoken")
else:
    print("ngrok authtoken configured!")

## 5. Start the Server

This cell will:
1. Start ngrok tunnel on port 8000
2. Print the public URL (copy this for your frontend)
3. Start the FastAPI server

**Note:** The first request will be slow as models are loaded into GPU memory.

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Allow nested event loops (required for Colab)
nest_asyncio.apply()

# Set ngrok authtoken
if NGROK_AUTHTOKEN:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("=" * 60)
print(f"ngrok tunnel established!")
print(f"")
print(f"PUBLIC URL: {public_url}")
print(f"")
print(f"Copy this URL and set it in your frontend/.env file:")
print(f'VITE_API_URL={public_url}')
print(f"")
print(f"Then restart your frontend: npm run dev")
print("=" * 60)

# Import and run the FastAPI app
import sys
sys.path.insert(0, '/content/final_year_project/backend')

from app.main import app

# Start the server (this will block)
uvicorn.run(app, host="0.0.0.0", port=8000)

## Troubleshooting

### ngrok errors
- Make sure you entered a valid authtoken
- Free ngrok accounts have limits (1 tunnel at a time)
- If you get "ERR_NGROK_108", close other ngrok tunnels

### Out of Memory (OOM)
- Use smaller models (SD instead of SDXL)
- Restart runtime and try again

### Slow first request
- Models are downloaded and loaded on first use
- SD Inpainting: ~2GB download, 30-60s load time
- Subsequent requests are fast

### Connection refused
- Wait for "Uvicorn running on http://0.0.0.0:8000" message
- Check that ngrok URL is correct in frontend .env
- Make sure frontend is rebuilt after changing .env

## Alternative: Run without ngrok (for testing)

If you just want to test the API without a frontend:

In [ ]:
# Test health endpoint
import requests

# Use the ngrok URL or localhost
API_URL = str(public_url) if 'public_url' in dir() else "http://localhost:8000"

try:
    response = requests.get(f"{API_URL}/api/health")
    print("Health check response:")
    print(response.json())
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the server is running (cell 5)")